In [1]:
# Scraping for Pi_ERP-BI

In [1]:
# Scraping prescription drugs' names
import urllib
import json
from bs4 import BeautifulSoup
import csv
import requests
import string
alpha = string.ascii_lowercase[:27]
drugs_result = []
for a in alpha:
    url = "https://www.everydayhealth.com/drugs/" + a
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    drugs = soup.find_all('div', attrs={'class': 'drug-class'})
    for drug in drugs:
        drugs_result.append({
            "name": drug.find('a').getText(),
            "link": drug.find('a').attrs.get("href")[7:]
        })

In [2]:
# Preparing filtered result set
drugs_final = []
for drug in drugs_result:
    if not "class" in drug.get("link"):
        drugs_final.append(drug)

In [5]:
# Scraping drugs interactions
for drug in drugs_final:
    r_interactions = []
    r_code = requests.get("https://rxnav.nlm.nih.gov/REST/rxcui?name=" + drug['link'])
    check_code = str(r_code.content)[str(r_code.content).find("rxnormId")+9:str(r_code.content).find("</rxnormId")]
    if represents_int(check_code):
        req_interactions_response = requests.get("https://rxnav.nlm.nih.gov/REST/interaction/interaction.json?rxcui=" + check_code)
        data = req_interactions_response.json()
        try:
            r_interactions = list(set([
                x['interactionConcept'][1]['minConceptItem']['name'].capitalize()
                for L in data["interactionTypeGroup"][0]['interactionType']
                for x in L['interactionPair']]
            ))
        except:
            r_interactions = []
    drug['interactions'] = r_interactions

In [ ]:
# Preparing label and constraints
from neo4jrestclient.client import GraphDatabase
db = GraphDatabase("http://localhost:7474", username="neo4j", password="veryveryhardpassword1234")
query = '''CREATE (d:Drug {name: "Initiated_label"})'''
db.query(query)
query = '''CREATE CONSTRAINT ON (d:Drug) ASSERT d.name IS UNIQUE'''
db.query(query)

In [8]:
# Database seeding
for drug in drugs_final:
    for d in drug['interactions']:
        db.query('''MERGE (d_init:Drug {name: "''' + drug['name'] + '''"})
        MERGE (d_fin:Drug {name: "''' + d + '''"}) MERGE (d_init)-[:INTERACTS_WITH]->(d_fin)''')

#### ________ Failed Trials ________

In [ ]:
# Create drug nodes
drug_label = db.labels.create("Drug").ipynb_checkpoints/
query = '''CREATE (d:Drug {name: "Initiated_label"})'''
db.query(query)
query = '''CREATE CONSTRAINT ON (d:Drug) ASSERT d.name IS UNIQUE'''
db.query(query)

In [126]:
for drug in drugs_final:
    url = "https://www.everydayhealth.com/drugs/" + drug.get("link")
    page = requests.get(url).text
    interactions = []
    soup = BeautifulSoup(page, 'html.parser')
    drugs = soup.find('div', attrs={'id': 'interactions', 'class': 'col-xs-12 col-sm-12 col-md-9 col-md-offset-3 collapseable'})
    if not drugs is None:
        inter_list = drugs.find_all('li')
        interactions = []
        for inter in inter_list:
            ll = inter.find('a')
            if not ll is None:
                interactions.append(ll.text.capitalize().split(" ")[0])
    drug['interactions'] = interactions

In [145]:
for drug in drugs_final:
    drug['interactions'] = list(set(drug['interactions']))

In [131]:
from neo4jrestclient.client import GraphDatabase
db = GraphDatabase("http://localhost:7474", username="neo4j", password="veryveryhardpassword1234")

In [ ]:
# Create drug nodes
drug_label = db.labels.create("Drug")
query = '''CREATE (d:Drug {name: "Initiated_label"})'''
db.query(query)
query = '''CREATE CONSTRAINT ON (d:Drug) ASSERT d.name IS UNIQUE'''
db.query(query)

In [153]:
for drug in drugs_final:
    #drug_label.add(d1)
    db.query('''MERGE (:Drug {name: "''' + drug['name'] + '''"})''')

In [ ]:
# Create interaction relationships
for drug in drugs_final:
    for d_interact in drug['interactions']:
        query = '''MERGE (d:Drug {name:"''' + drug['name'] + '''"})-[:INTERACTS_WITH]->(:Drug {name:"''' + d_interact + '''"})'''
        db.query(query)

In [289]:
r_code = requests.get("https://rxnav.nlm.nih.gov/REST/rxcui?name=paracetamol")
represents_int(str(r_code.content)[str(r_code.content).find("rxnormId")+9:str(r_code.content).find("</rxnormId")])

True

In [224]:
r_interactions = requests.get("https://rxnav.nlm.nih.gov/REST/interaction/interaction.json?rxcui=214178")

In [259]:
import json
from pprint import pprint
data = r_interactions.json()
pprint(len(data["interactionTypeGroup"][0]['interactionType']))

2


In [4]:
def represents_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [ ]:
pprint(list(set([x['interactionConcept'][1]['minConceptItem']['name'].capitalize() for L in data["interactionTypeGroup"][0]['interactionType'] for x in L['interactionPair']])))